In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .config("spark.driver.memory", "15g") \
        .appName('Twitter Sentiment Analysis') \
        .getOrCreate() \
       
print('Session created')
sc = spark.sparkContext

Session created


In [0]:
from pyspark import SparkConf, SparkContext
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
!pip install pycountry
!pip install wordcloud
!pip install textblob
!pip install pyfs

  Using cached pycountry-22.3.5.tar.gz (10.1 MB)
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
    Preparing wheel metadata ... - done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681829 sha256=d1471255e8843192113483c0f98b402d50fcd56e5d38f78c71b0ee91361249e0
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
     |████████████████████████████████| 1.5 MB 5.6 MB/s 
     |████████████████████████████████| 78 kB 10.6 MB/s 
     |████████████████████████████████| 96 kB 8.0 MB/s 
     |████████████████████████████████| 764 kB 25.1 

In [0]:
!pip install langdetect

     |████████████████████████████████| 981 kB 6.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=c004386ccdcd4f3e1bdb532520f75db4f50771a12af67ca73730a7db525e361e
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import os
import nltk
import pycountry
import re
import string
import nltk
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
def mount_s3_bucket(access_key, secret_key, bucket_name, mount_folder):
  ACCESS_KEY_ID = access_key
  SECRET_ACCESS_KEY = secret_key
  ENCODED_SECRET_KEY = SECRET_ACCESS_KEY.replace("/", "%2F")

  print ("Mounting", bucket_name)

  try:
    # Unmount the data in case it was already mounted.
    dbutils.fs.unmount("/mnt/%s" % mount_folder)
    
  except:
    # If it fails to unmount it most likely wasn't mounted in the first place
    print ("Directory not unmounted: ", mount_folder)
    
  finally:
    # Lastly, mount our bucket.
    dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY_ID, ENCODED_SECRET_KEY, bucket_name), "/mnt/%s" % mount_folder)
    #dbutils.fs.mount("s3a://"+ ACCESS_KEY_ID + ":" + ENCODED_SECRET_KEY + "@" + bucket_name, mount_folder)
    print ("The bucket", bucket_name, "was mounted to", mount_folder, "\n")
    

In [0]:
# Set AWS programmatic access credentials
ACCESS_KEY = ""
SECRET_ACCESS_KEY = ""

In [0]:
mount_s3_bucket(ACCESS_KEY, SECRET_ACCESS_KEY, 'twitter-project-wcd/twitter/2022/05/19','twitter-proj')


Mounting twitter-project-wcd/twitter/2022/05/19
/mnt/twitter-proj has been unmounted.
The bucket twitter-project-wcd/twitter/2022/05/19 was mounted to twitter-proj 



In [0]:
%fs ls /mnt/twitter-proj/

path,name,size,modificationTime
dbfs:/mnt/twitter-proj/02/,02/,0,0
dbfs:/mnt/twitter-proj/03/,03/,0,0
dbfs:/mnt/twitter-proj/04/,04/,0,0
dbfs:/mnt/twitter-proj/05/,05/,0,0


In [0]:
#credit to databricks streaming/static amazon kinesis demo notebook for the help!

InputPath = "/mnt/twitter-proj/*" #the star is a wildcard, this means everytime I run this notebook it will query more data!

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType(
  [StructField("data", StructType(
    [StructField('id', StringType(), True), StructField('text', StringType(), True)]
  ))]
)

# Static DataFrame representing data in the JSON files.
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(InputPath)
    .select("data.text")
)

display(staticInputDF)

text
@wtf1official Laguna Seca! https://t.co/5bOlpq0FHP
"@AlexFrazier1221 @cybertrucknana @DineshDSouza @kelliwardaz 2008 was the last time GA had a race for president and senate on the same ballot. Back then, there were 172k more votes for president than senate, or 4.38% of votes cast. McCain won GA’s electoral college votes in 2008. Should they have gone to Obama? https://t.co/EaimKvz7L2"
@SKeshel yep
"When the weather isn’t crazy hot or humid yet, we take advantage and practice our multiplication fact fluency outside relay race style! #WONDERfulRES https://t.co/fX75KqLZBr"
Social region best prevent yes return ability television race try continue. https://t.co/hqDTgpAlTP
"The race today was won by 29m: 🟩 14:57 @HLN_BE: Onweer op komst met mogelijk hagel en.… 🟥 15:26 @gva: Vandaag nog 28 graden, maar dan onweer o.… 🟥 15:26 @Nieuwsblad_be: Vanaf vannacht code geel voor onweer "".…"
@ford_nik @IvvyNemo I'm curious whether you lived in Ukraine for an extensive period of time recently & integrated into society?
@RevStatus @LyibovPop @RTSG_Prism_ General contractor who owns a palacial McMansion on a 20-acre lot and has 3 lifted pickups: I am combatting liberalism by being racist
"@michelle_byoung This. Right. Here. I will support each and every Democrat running for office, except him. IDGAMF what his policies are. He held an innocent unarmed Black man hostage with a loaded shotgun and NEVER apologized for it. That's a deal breaker for me."
LADIES – START YOUR ENGINES! #2022 BUILD TRAIN RACE STARTS.: LADIES – START YOUR ENGINES! ROYAL ENFIELD BTR ROAD RACING KICKS OFF AT VIR Fifteen women ready for #2022 BUILD. TRAIN. RACE. opening round this weekend at Virginia International Raceway.… https://t.co/soQB34mZbO


In [0]:
staticInputDF = staticInputDF.dropDuplicates()
staticInputDF.count() #sanity check

Out[40]: 114563

In [0]:
# tweet_list = staticInputDF.to_pandas_on_spark()
tweet_list1 = staticInputDF.select("*").toPandas()

In [0]:
tw_list = pd.DataFrame(tweet_list1)
print(tw_list)


                                                     text
0       @Tete_delacourse Haven't had too many bad inju...
1       @elonmusk Right &amp; Left... All Bad! We need...
2                        @formulaanas Yeeeesss \nAt last!
3       Matt, your racism is showing... https://t.co/A...
4                            @typo_cat @fiji2002 Bastard!
...                                                   ...
114558  @avixkook they’ll probably be like “it’s just ...
114559  You gotta run your own race. No matter what pl...
114560  Social region best prevent yes return ability ...
114561  @AshleyroweWKBW It's always interesting to see...
114562  @Sonnysboy319 I don't hate her, but was being ...

[114563 rows x 1 columns]


In [0]:
values = tw_list
tw_list["0"] = values
tw_list

,text,0
0,@Tete_delacourse Haven't had too many bad inju...,@Tete_delacourse Haven't had too many bad inju...
1,@elonmusk Right &amp; Left... All Bad! We need...,@elonmusk Right &amp; Left... All Bad! We need...
2,@formulaanas Yeeeesss \nAt last!,@formulaanas Yeeeesss \nAt last!
3,"Matt, your racism is showing... https://t.co/A...","Matt, your racism is showing... https://t.co/A..."
4,@typo_cat @fiji2002 Bastard!,@typo_cat @fiji2002 Bastard!
...,...,...
114558,@avixkook they’ll probably be like “it’s just ...,@avixkook they’ll probably be like “it’s just ...
114559,You gotta run your own race. No matter what pl...,You gotta run your own race. No matter what pl...
114560,Social region best prevent yes return ability ...,Social region best prevent yes return ability ...
114561,@AshleyroweWKBW It's always interesting to see...,@AshleyroweWKBW It's always interesting to see...


In [0]:
#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: ',"" ,str(x))
rt = lambda x: re.sub('[^A-Za-z0-9]+', ' ',str(x))
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head(10)

,text,0
0,tete delacourse haven t had too many bad inju...,@Tete_delacourse Haven't had too many bad inju...
1,elonmusk right amp left all bad we need to ge...,@elonmusk Right &amp; Left... All Bad! We need...
2,formulaanas yeeeesss at last,@formulaanas Yeeeesss \nAt last!
3,matt your racism is showing https t co aa2dhue...,"Matt, your racism is showing... https://t.co/A..."
4,typo cat fiji2002 bastard,@typo_cat @fiji2002 Bastard!
5,johnhmcwhorter proposes an interesting and th...,@JohnHMcWhorter proposes an interesting and th...
6,bbwave2020 hottatatta catturd2 how much were ...,@BBWave2020 @HottaTatta @catturd2 How much wer...
7,just yea this article continues to analyze all...,Just yea this article continues to analyze all...
8,gracias and as a trained female mixed race his...,"Gracias.\nAnd, as a trained, female, mixed rac..."
9,gmbutts nytimes https t co 4ugnurcjql,@gmbutts @nytimes https://t.co/4UGnurcjql


In [0]:
def get_polarity(text):
  return TextBlob(text).sentiment.polarity
tw_list['Polarity'] = tw_list['text'].apply(get_polarity)

In [0]:
tw_list['Sentiment_Type']=''
tw_list.loc[tw_list.Polarity>0,'Sentiment_Type']='POSITIVE'
tw_list.loc[tw_list.Polarity==0,'Sentiment_Type']='NEUTRAL'
tw_list.loc[tw_list.Polarity<0,'Sentiment_Type']='NEGATIVE'

In [0]:
tw_list.count() #now we have information to work with! we have text, a score and sentiment. model time!

Out[47]: text              114563
0                 114562
Polarity          114563
Sentiment_Type    114563
dtype: int64

In [0]:
type(tw_list)

Out[48]: pandas.core.frame.DataFrame

In [0]:
cleaned_tweets =  spark.createDataFrame(tw_list)

In [0]:
cleaned_tweets.count()

Out[50]: 114563

In [0]:
type(cleaned_tweets)

Out[51]: pyspark.sql.dataframe.DataFrame

In [0]:
#cache tweets for faster iteration
cleaned_tweets.cache()
# run the count action to materialize the cache
cleaned_tweets.count()

Out[52]: 114563

In [0]:
from pyspark.ml.feature import NGram, VectorAssembler, StopWordsRemover, HashingTF, IDF, Tokenizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer

# Use 90% cases for training, 10% cases for testing
train, test = cleaned_tweets.randomSplit([0.9, 0.1], seed=20200819)

# Create transformers for the ML pipeline
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")
cv = CountVectorizer(vocabSize=2**16, inputCol="filtered", outputCol='cv')
idf = IDF(inputCol='cv', outputCol="1gram_idf", minDocFreq=5) #minDocFreq: remove sparse terms
assembler = VectorAssembler(inputCols=["1gram_idf"], outputCol="features")
label_encoder = StringIndexer(inputCol = "Sentiment_Type", outputCol = "label")
lr = LogisticRegression(maxIter=150)
pipeline = Pipeline(stages=[tokenizer, stopword_remover, cv, idf, assembler, label_encoder, lr])

pipeline_model = pipeline.fit(train)
predictions = pipeline_model.transform(test)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(test.count())
roc_auc = evaluator.evaluate(predictions)

print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))

Accuracy Score: 0.7891
ROC-AUC: 0.7888


In [0]:
predictions = predictions.drop("0", "tokens", "filtered", "cv", "1gram_idf", "features", "rawPrediction", "probability")

In [0]:
mount_s3_bucket(ACCESS_KEY, SECRET_ACCESS_KEY, 'twitter-out-final/output/','twitter-output2')

In [0]:
predictions.coalesce(1).write.csv("mnt/twitter-output2/predictions.csv")

In [0]:
%fs ls /mnt/twitter-output2/

path,name,size,modificationTime
dbfs:/mnt/twitter-output2/predictions1.parquet/,predictions1.parquet/,0,0


In [0]:
display(predictions)

text,Polarity,Sentiment_Type,label,prediction
100 gatlin sanath07 how dare you say i m a racist when you are not even ready to accept the facts i am tamil and have seen my own people were killed in thousands and if i talk about it am i going to be called racist how funny,0.21250000000000002,POSITIVE,0.0,2.0
1zzyzyx1 this is wht biden america like we have to stop this vote red vote red fjb,0.0,NEUTRAL,1.0,1.0
2000mules https t co pauafxygrz,0.0,NEUTRAL,1.0,1.0
2022 05 18 https t co waz7xscgtr 100 100,0.0,NEUTRAL,1.0,1.0
2022 05 19 8r 12 14 1 7 01 2 7 00 3 7 04 4 6 97 6 7 02 5 7 01 https t co uliyeqhk7b,0.0,NEUTRAL,1.0,1.0
23mjoshi white nationalist don t confuse american response and solution to islamic aggression with racism in america all people are created equally and ethnicity does not matter but ideas and character does,0.0,NEUTRAL,1.0,2.0
503i7 looking at how us military basis masquerading as nations israel amp ukraine are knee deep in racism i wonder what will happen to taiwanese society once us is done emboldening certain elements in that nation,0.03809523809523809,POSITIVE,0.0,2.0
54battalion the fone is a scapegoat by the teacher whose deep contention underlying factor is the use a different language by the kid hence her shouting is driven by egocentric racist attitude provoking the reaction from the student teacher failed to handle the situation professionally,-0.1,NEGATIVE,2.0,1.0
55bailey repmattgaetz fuck no there s not,-0.4,NEGATIVE,2.0,2.0
937luke nonno paula jfistacuffs i am wf3 alas racism makes me super miserable others not so much i guess,-0.21666666666666667,NEGATIVE,2.0,2.0
